In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score 
import xgboost as xgb

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

import seaborn as sns
import lightgbm as lgb
import random

import shap
import joblib

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Load the dataset
# We assume that the dataset is in EXCEL format
file_path = 'CLEANED_FILE_INPUT.xlsx'  # Enter the file path
df = pd.read_excel(file_path)


print(df.head())
print(df.info())


In [ ]:
exp = 16
output_dir = 'DIRECTORY/'+str(exp)

In [ ]:
# Converting time variables to datetime format
# Change to handle 'DD/MM/YYYY HH:MM' format
df['INGRESSOSALA'] = pd.to_datetime(df['INGRESSOSALA'], format='%d/%m/%Y %H:%M')
df['USCITASALA'] = pd.to_datetime(df['USCITASALA'], format='%d/%m/%Y %H:%M')

# Creating 'DURATION' column (in minutes)
df['DURATA'] = (df['USCITASALA'] - df['INGRESSOSALA']).dt.total_seconds() / 60

# Remove lines with negative duration (any errors)
df = df[df['DURATA'] > 0]

In [ ]:
df['DURATA']

In [ ]:
# Null Analysis
missing_data = df.isnull().sum()

# Remove columns with more than 50% missing values
threshold = 0.5
df = df[df.columns[df.isnull().mean() < threshold]]

# Check for no more null values
df = df.dropna(how='any',axis=0)


In [ ]:
# Use Seaborn to plot the point distribution
plt.figure(figsize=(8, 6))
sns.histplot(df['DURATA'], kde=True, bins=10, color='skyblue')

# Set titles
plt.title('Distribution of Points')
plt.xlabel('Duration')
plt.ylabel('Items')

# Show graph
plt.show()

In [ ]:
# Plot histogram of durations
plt.figure(figsize=(10, 6))
plt.hist(df['DURATA'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of the duration of the interventions', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=14)
plt.ylabel('Frequence', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Boxplot of durations
plt.figure(figsize=(8, 6))
plt.boxplot(df['DURATA'], vert=False, patch_artist=True, 
            boxprops=dict(facecolor='lightblue', color='blue'),
            whiskerprops=dict(color='blue'), capprops=dict(color='blue'),
            medianprops=dict(color='red'))
plt.title('Boxplot of the duration of the interventions', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=14)
plt.show()


In [ ]:
# Display remaining columns in dataset
print("Remaining columns in dataset:")
print(df.columns)
X = df
X = X.drop(columns=X.select_dtypes(include='datetime64[ns]').columns)
print(X.dtypes)

progressivo = df['PROGRESSIVO']  # Keep column name
rep = df['REPARTO']  # Keep column name


In [ ]:
def remove_high_corr_features_with_categoricals(df, threshold=0.95):
    df = df.copy()
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    num_cols = df.select_dtypes(exclude=['object', 'category']).columns

    # Encode categorical features
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    df[cat_cols] = encoder.fit_transform(df[cat_cols])

    # Compute correlation matrix (absolute values)
    corr_matrix = df.corr().abs()

    # Create a boolean mask to ignore the upper triangle (it's symmetric)
    upper = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    upper_matrix = corr_matrix.where(upper)

    # Find columns with correlation > threshold
    to_drop = [column for column in upper_matrix.columns if any(upper_matrix[column] > threshold)]

    # Drop them from the dataframe
    df_filtered = df.drop(columns=to_drop)

    return df_filtered, to_drop

In [ ]:
df_cleaned, removed = remove_high_corr_features_with_categoricals(df.drop(columns=['DURATA']))
print("Removed features due to high correlation:", removed)


In [ ]:

# 'DURATA' represents target variable
y = df['DURATA']
X.drop(columns=['DURATA'], inplace=True)
X = df_cleaned

In [ ]:
#experiment ID
exp = 15

rf = RandomForestRegressor(n_estimators=10, random_state=SEED)
gb = GradientBoostingRegressor(n_estimators=400, learning_rate=0.01, max_depth=15, random_state=SEED)
dt = DecisionTreeRegressor(criterion='friedman_mse', max_depth=50, min_samples_split=2, random_state=SEED)
xgboost = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=400, learning_rate=0.1, max_depth=5,
                           tree_method="hist", eval_metric='mae', random_state=SEED)
svr = SVR()
knn = KNeighborsRegressor(n_neighbors=5, weights='distance')

In [ ]:
param_grids = {
    'Random Forest': {
        'n_estimators': [2, 5, 10],
        'max_depth': [ 2, 4, None],
         'min_samples_split': [2, 5]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200, 400],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [15, 30],
    },
    'Decision Tree': {
        'max_depth': [50, 100],
        'min_samples_split': [1, 2],
        'criterion': ['friedman_mse']
    },
    'Support Vector Regressor': {
        'C': [0.1, 1],
        'epsilon': [0.01, 0.1],
        'kernel': ['rbf', 'linear']
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 4, 5, 32],
        'weights': ['uniform', 'distance']
    },
    'xgboost': {
        'n_estimators': [200, 400, 600],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [2, 5]
    }
}


In [ ]:
# Split data into training and test sets
# Split data into training and test sets while keeping PROGRESSIVO
num_bins = 170  # You can adjust this
y_binned = pd.qcut(y, q=num_bins, labels=False, duplicates='drop')

# Use the binned variable for stratification
X_train, X_test, y_train, y_test, prog_train, prog_test, rep_train, rep_test = train_test_split(
    X, y, progressivo, rep,
    test_size=0.2,
    random_state=SEED,
    stratify=y_binned
)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to DataFrame and re-associate the PROGRESSIVE column in the test set
X_test_df = pd.DataFrame(X_test_scaled)
X_test_df['PROGRESSIVO'] = prog_test.values  # Let's re-associate PROGRESSIVO
X_test_df['REPARTO'] = rep_test.values  # Let's re-associate PROGRESSIVO

y_test_df = pd.DataFrame(y_test)
y_test_df['PROGRESSIVO'] = prog_test.values  # Let's re-associate PROGRESSIVO
y_test_df['REPARTO'] = rep_test.values  # Let's re-associate REPARTO

# Convert to DataFrame and re-associate the PROGRESSIVE column in the train set
X_train_df = pd.DataFrame(X_train_scaled)
X_train_df['PROGRESSIVO'] = prog_train.values  # Let's re-associate PROGRESSIVO
X_train_df['REPARTO'] = rep_train.values  # Let's re-associate REPARTO

y_train_df = pd.DataFrame(y_train)
y_train_df['PROGRESSIVO'] = prog_train.values  # Let's re-associate PROGRESSIVO
y_train_df['REPARTO'] = rep_train.values  # Let's re-associate REPARTO

# List of models 
models = {
    'Random Forest': rf,
    'Gradient Boosting': gb,
    'Decision Tree': dt,
    'Support Vector Regressor': svr,
    'K-Nearest Neighbors': knn,
    'xgboost': xgboost

}

# Prepare data for the DataFrame
data = []
for name, model in models.items():
    params = model.get_params()
    for param, value in params.items():
        data.append({
            'Model': name,
            'Parameter': param,
            'Value': value
        })


os.makedirs(output_dir, exist_ok=True)


# Create and save the DataFrame
df = pd.DataFrame(data)
df.to_csv(output_dir+'/configurations.csv', index=False)


best_models = {}
preds = {}
mae_results, rmse_results, r2_results = {}, {}, {}

for name, model in models.items():
    print(f"Training {name} with GridSearchCV...")

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        cv=5,  # CV is only on training set
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=1
    )

    # Fit only on training data
    grid_search.fit(X_train_scaled, y_train)

    best_model = grid_search.best_estimator_
    best_models[name] = best_model

    # Predict on the untouched test set
    y_pred = best_model.predict(X_test_scaled)
    preds[name] = y_pred

    # Evaluate
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    mae_results[name] = mae
    rmse_results[name] = rmse
    r2_results[name] = r2

    # Print results
    print(f"{name} - Best Params: {grid_search.best_params_}")
    print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.2f}")

    # Save model
    joblib.dump(best_model, f'../models/no_out/{exp}/model_durata_intervento_{name}_best.pkl')


In [ ]:
merged_pred_y = pd.merge(y_test_df['PROGRESSIVO'], pd.DataFrame(y_pred, columns=['LABEL']), left_index=True, right_index=True)
merged_pred_y_2 = pd.merge(y_test_df['REPARTO'], merged_pred_y, left_index=True, right_index=True)

In [ ]:
merged_pred = pd.merge(X_test_df['PROGRESSIVO'], pd.DataFrame(y_pred, columns=['PREDICTED']), left_index=True, right_index=True)
merged_pred_2 = pd.merge(X_test_df['REPARTO'], merged_pred, left_index=True, right_index=True)

In [ ]:
merged_pred_train = pd.merge(X_train_df['PROGRESSIVO'], pd.DataFrame(np.asanyarray(y_train), columns=['LABEL']), left_index=True, right_index=True)
merged_pred_train_2 = pd.merge(X_train_df['REPARTO'], merged_pred_train, left_index=True, right_index=True)

In [ ]:
# Plot histogram of durations
plt.figure(figsize=(10, 6))
plt.hist(merged_pred_y_2['LABEL'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of the duration of the interventions TEST', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=14)
plt.ylabel('Frequence', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Boxplot of durations
plt.figure(figsize=(8, 6))
plt.boxplot(merged_pred_y_2['LABEL'], vert=False, patch_artist=True, 
            boxprops=dict(facecolor='lightblue', color='blue'),
            whiskerprops=dict(color='blue'), capprops=dict(color='blue'),
            medianprops=dict(color='red'))
plt.title('Boxplot of the duration of the interventions TEST', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=14)
plt.show()

In [ ]:
# Plot histogram of durations
plt.figure(figsize=(10, 6))
plt.hist(merged_pred_train['LABEL'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of the duration of the interventions TRAIN', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=14)
plt.ylabel('Frequence', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Boxplot of durations
plt.figure(figsize=(8, 6))
plt.boxplot(merged_pred_train['LABEL'], vert=False, patch_artist=True, 
            boxprops=dict(facecolor='lightblue', color='blue'),
            whiskerprops=dict(color='blue'), capprops=dict(color='blue'),
            medianprops=dict(color='red'))
plt.title('Boxplot of the duration of the interventions TRAIN', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=14)
plt.show()

In [ ]:
# Convert results to lists for plotting
model_names = list(mae_results.keys())
mae_values = list(mae_results.values())
rmse_values = list(rmse_results.values())
r2_values = list(r2_results.values())

# Create the graph of MAE
plt.figure(figsize=(10, 6))
plt.barh(model_names, mae_values, color='skyblue')
plt.xlabel('MAE (minutes)')
plt.title('Mean Absolute Error per Modello')
plt.savefig(output_dir+'/MAE.png', dpi=300)
plt.show()
plt.close()

# Create the graph of RMSE
plt.figure(figsize=(10, 6))
plt.barh(model_names, rmse_values, color='lightgreen')
plt.xlabel('RMSE (minutes)')
plt.title('Root Mean Squared Error per Modello')
plt.savefig(output_dir+'/RMSE.png', dpi=300)
plt.show()
plt.close()

# Create the graph of RMSE
plt.figure(figsize=(10, 6))
plt.barh(model_names, r2_values, color='gray')
plt.xlabel('R2 (minutes)')
plt.title('Coefficient of Determination (R² Score) per Modello')
plt.savefig(output_dir+'/R2.png', dpi=300)
plt.show()
plt.close()



In [ ]:
# Prepare dataframe with these columns: PROGRESSIVO  Pred	Ground Truth (DURATA)

# Create a DataFrame
df = pd.DataFrame({"PROGRESSIVO ":prog_test.values,"Pred": y_pred, "Ground Truth": y_test})

# Save to an Excel file
df.to_excel(output_dir+'/output.xlsx', index=False)

In [ ]:
interventi, id_predictions, y_rep = merged_pred_2, merged_pred_2['PROGRESSIVO'], merged_pred_y

In [ ]:
merged_pred_y

In [ ]:
def split_by_hospital(interventi, id_predictions, y_pred):
    # Merge predicted labels into the interventi dataframe based on PROGRESSIVO
    interventi_merged = interventi.merge(y_pred, on="PROGRESSIVO", how="inner")

    sanremo_df = pd.DataFrame()
    imperia_df = pd.DataFrame()
    bordighera_df = pd.DataFrame()

    for id in id_predictions:
        row = interventi_merged.loc[interventi_merged["PROGRESSIVO"] == id]
        if row.empty:
            continue

        reparto = row["REPARTO"].values[0]

        if "SANREMO" in reparto:
            sanremo_df = pd.concat([sanremo_df, row])
        elif "IMPERIA" in reparto:
            imperia_df = pd.concat([imperia_df, row])
        elif "BORDIGHERA" in reparto:
            bordighera_df = pd.concat([bordighera_df, row])

    return sanremo_df.reset_index(drop=True), imperia_df.reset_index(drop=True), bordighera_df.reset_index(drop=True)


In [ ]:
#split prediction for each hospital to further analyses
sanremo, imperia, bordighera = split_by_hospital(interventi, id_predictions, y_rep)

In [ ]:
sanremo

In [ ]:
imperia["ERROR"] = imperia["LABEL"] - imperia["PREDICTED"]
sanremo["ERROR"] = sanremo["LABEL"] - sanremo["PREDICTED"]
bordighera["ERROR"] = bordighera["LABEL"] - bordighera["PREDICTED"]


In [ ]:
imperia

In [ ]:
imperia = imperia.sort_values(by='ERROR')
bordighera = bordighera.sort_values(by='ERROR')
sanremo = sanremo.sort_values(by='ERROR')


In [ ]:
name = ['Sanremo', 'Imperia', 'Bordighera']

In [ ]:
for v in name:
    if (v == 'Sanremo'):
        i = sanremo
    elif(v == 'Imperia'):
        i = imperia
    elif(v == 'Bordighera'):
        i = bordighera

    y_pred_osp = np.asarray(i['PREDICTED'])
    y_test_osp = np.asarray(i['LABEL'])
    indices = list(range(len(y_test_osp)))

    plt.figure(figsize=(26, 6))

    # Predictions
    plt.plot(
        indices,
        y_pred_osp,
        label='Predictions',
        color='blue',
        linewidth=1.5,
        marker='o',
        markersize=5,
        alpha=0.6,
        zorder=2
    )

    # Real Value
    plt.plot(
        indices,
        y_test_osp,
        label='Real Value',
        color='brown',
        linewidth=1.5,
        marker='x',
        markersize=5,
        alpha=0.6,
        zorder=3
    )

    plt.title('Comparison between Actual and Predicted Values in '+str(v)+' (Patients sorted by error)', fontsize=20)
    plt.xlabel('Index', fontsize=14)
    plt.ylabel('Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.savefig(output_dir+'/Comparison between Actual and Predicted Values in '+str(v)+' (Patients sorted by error.png', dpi=300)
    plt.show()
    plt.close()

    plt.figure(figsize=(26, 6))

    # Predictions vs real value
    plt.plot(
        indices,
        y_test_osp - y_pred_osp,
        label='y_test-y_pred',
        color='blue',
        linewidth=1.5,
        marker='o',
        markersize=5,
        alpha=0.6,
        zorder=2
    )


    plt.title('Difference between Actual and Predicted Values in '+str(v)+' (Patients sorted by error)', fontsize=20)
    plt.xlabel('Index', fontsize=14)
    plt.ylabel('Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.savefig(output_dir+'/Difference between Actual and Predicted Values in '+str(v)+' (Patients sorted by error.png', dpi=300)
    plt.show()
    plt.close()

    # groupby error for each department
    grouped = i.groupby('REPARTO')['ERROR'].apply(list)

    # Boxplot
    plt.figure(figsize=(12, 6))
    plt.boxplot(grouped.values, labels=grouped.index, vert=True)
    plt.title("Error Distribution by Department in "+str(v))
    plt.ylabel("Error")
    plt.xticks(rotation=45, ha='right')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(output_dir+'/Error Distribution by Department in '+str(v)+'.png', dpi=300)
    plt.show()
    plt.close()

In [ ]:
mae_results

# SHAP ANALYSIS

In [ ]:
feature_names = X.columns

In [ ]:
feature_names

In [ ]:
X_train_scaled.shape

In [ ]:
X_test_df_shap = pd.DataFrame(X_test_scaled, columns=feature_names)

In [ ]:

# Create the TreeExplainer
explainer = shap.Explainer(best_model.predict, X_test_df_shap)

# Calculate SHAP values for the test set (or any data)
shap_values = explainer(X_test_df_shap)
shap.summary_plot(shap_values, X_test_df_shap)